<a href="https://colab.research.google.com/github/Ducktard/Text2KnowledgeGraph/blob/dev/text3KnowledgeGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning into 'Text2KnowledgeGraph'...


#hat geklappt

In [ ]:
Änderung by Alex